In [1]:
import xgboost as xgb
import numpy as np
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
import time

In [2]:
%%time
# Fetch dataset using sklearn
cov = fetch_covtype()
X = cov.data
y = cov.target

CPU times: user 1min 15s, sys: 1.13 s, total: 1min 16s
Wall time: 1min 19s


In [3]:
%%time
# Create 0.75/0.25 train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75,
                                                    random_state=42)

CPU times: user 180 ms, sys: 168 ms, total: 348 ms
Wall time: 346 ms


In [13]:
# Specify sufficient boosting iterations to reach a minimum
num_round = 3000

# Leave most parameters as default
param = {'objective': 'multi:softmax', # Specify multiclass classification
         'num_class': 8, # Number of possible output classes
         'tree_method': 'gpu_hist' # Use GPU accelerated algorithm
         }

In [14]:
%%time
# Convert input data from numpy to XGBoost format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

CPU times: user 144 ms, sys: 72 ms, total: 216 ms
Wall time: 215 ms


In [ ]:
%%time
gpu_res = {} # Store accuracy result
tmp = time.time()
# Train model
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=gpu_res)
print("GPU Training Time: %s seconds" % (str(time.time() - tmp)))

[0]	test-merror:0.254804
[1]	test-merror:0.247892
[2]	test-merror:0.244263
[3]	test-merror:0.240683
[4]	test-merror:0.23848
[5]	test-merror:0.234777
[6]	test-merror:0.232154
[7]	test-merror:0.22973
[8]	test-merror:0.227169
[9]	test-merror:0.224629
[10]	test-merror:0.222639
[11]	test-merror:0.220773
[12]	test-merror:0.218453
[13]	test-merror:0.215582
[14]	test-merror:0.214605
[15]	test-merror:0.21223
[16]	test-merror:0.211169
[17]	test-merror:0.209868
[18]	test-merror:0.208622
[19]	test-merror:0.205917
[20]	test-merror:0.20434
[21]	test-merror:0.203727
[22]	test-merror:0.202591
[23]	test-merror:0.201621
[24]	test-merror:0.199817
[25]	test-merror:0.199025
[26]	test-merror:0.198619
[27]	test-merror:0.197566
[28]	test-merror:0.195363
[29]	test-merror:0.194474
[30]	test-merror:0.193662
[31]	test-merror:0.193235
[32]	test-merror:0.191672
[33]	test-merror:0.190103
[34]	test-merror:0.189621
[35]	test-merror:0.188685
[36]	test-merror:0.186296
[37]	test-merror:0.185945
[38]	test-merror:0.185827


[309]	test-merror:0.07184
[310]	test-merror:0.071778
[311]	test-merror:0.071627
[312]	test-merror:0.071351
[313]	test-merror:0.071331
[314]	test-merror:0.071145
[315]	test-merror:0.070911
[316]	test-merror:0.070759
[317]	test-merror:0.07058
[318]	test-merror:0.07047
[319]	test-merror:0.070339
[320]	test-merror:0.070078
[321]	test-merror:0.07003
[322]	test-merror:0.069995
[323]	test-merror:0.06983
[324]	test-merror:0.069844
[325]	test-merror:0.06961
[326]	test-merror:0.069355
[327]	test-merror:0.069238
[328]	test-merror:0.069217
[329]	test-merror:0.0691
[330]	test-merror:0.068873
[331]	test-merror:0.068715
[332]	test-merror:0.068522
[333]	test-merror:0.068501
[334]	test-merror:0.068425
[335]	test-merror:0.068425
[336]	test-merror:0.068391
[337]	test-merror:0.06826
[338]	test-merror:0.068253
[339]	test-merror:0.068171
[340]	test-merror:0.068026
[341]	test-merror:0.067861
[342]	test-merror:0.067716
[343]	test-merror:0.067489
[344]	test-merror:0.067379
[345]	test-merror:0.067255
[346]	test

[614]	test-merror:0.049032
[615]	test-merror:0.048942
[616]	test-merror:0.048894
[617]	test-merror:0.048749
[618]	test-merror:0.048708
[619]	test-merror:0.048529
[620]	test-merror:0.048385
[621]	test-merror:0.048378
[622]	test-merror:0.048426
[623]	test-merror:0.048502
[624]	test-merror:0.048433
[625]	test-merror:0.048371
[626]	test-merror:0.048343
[627]	test-merror:0.048254
[628]	test-merror:0.048226
[629]	test-merror:0.048309
[630]	test-merror:0.048199
[631]	test-merror:0.048171
[632]	test-merror:0.048109
[633]	test-merror:0.048013
[634]	test-merror:0.047937
[635]	test-merror:0.047916
[636]	test-merror:0.047923
[637]	test-merror:0.047882
[638]	test-merror:0.047841
[639]	test-merror:0.047827
[640]	test-merror:0.047779
[641]	test-merror:0.047744
[642]	test-merror:0.047786
[643]	test-merror:0.047751
[644]	test-merror:0.047772
[645]	test-merror:0.047744
[646]	test-merror:0.047751
[647]	test-merror:0.047737
[648]	test-merror:0.047627
[649]	test-merror:0.047669
[650]	test-merror:0.047648
[

[919]	test-merror:0.041108
[920]	test-merror:0.041149
[921]	test-merror:0.04108
[922]	test-merror:0.041059
[923]	test-merror:0.04108
[924]	test-merror:0.04108
[925]	test-merror:0.041039
[926]	test-merror:0.041053
[927]	test-merror:0.041032
[928]	test-merror:0.041011
[929]	test-merror:0.04097
[930]	test-merror:0.040949
[931]	test-merror:0.040942
[932]	test-merror:0.04086
[933]	test-merror:0.040853
[934]	test-merror:0.040818
[935]	test-merror:0.040832
[936]	test-merror:0.040825
[937]	test-merror:0.040784
[938]	test-merror:0.040818
[939]	test-merror:0.040825
[940]	test-merror:0.040818
[941]	test-merror:0.04075
[942]	test-merror:0.040722
[943]	test-merror:0.040633
[944]	test-merror:0.04066
[945]	test-merror:0.040626
[946]	test-merror:0.040612
[947]	test-merror:0.040639
[948]	test-merror:0.040591
[949]	test-merror:0.040577
[950]	test-merror:0.040612
[951]	test-merror:0.040612
[952]	test-merror:0.040522
[953]	test-merror:0.040543
[954]	test-merror:0.040522
[955]	test-merror:0.040522
[956]	te

[1217]	test-merror:0.037066
[1218]	test-merror:0.037101
[1219]	test-merror:0.037066
[1220]	test-merror:0.03708
[1221]	test-merror:0.03708
[1222]	test-merror:0.036977
[1223]	test-merror:0.036949
[1224]	test-merror:0.037018
[1225]	test-merror:0.037004
[1226]	test-merror:0.037101
[1227]	test-merror:0.036956
[1228]	test-merror:0.037039
[1229]	test-merror:0.036991
[1230]	test-merror:0.036977
[1231]	test-merror:0.03697
[1232]	test-merror:0.037004
[1233]	test-merror:0.037011
[1234]	test-merror:0.036977
[1235]	test-merror:0.036984
[1236]	test-merror:0.037018
[1237]	test-merror:0.037039
[1238]	test-merror:0.036977
[1239]	test-merror:0.036977
[1240]	test-merror:0.036977
[1241]	test-merror:0.036998
[1242]	test-merror:0.037018
[1243]	test-merror:0.036949
[1244]	test-merror:0.036956
[1245]	test-merror:0.036956
[1246]	test-merror:0.036908
[1247]	test-merror:0.036874
[1248]	test-merror:0.036812
[1249]	test-merror:0.036777
[1250]	test-merror:0.036798
[1251]	test-merror:0.036825
[1252]	test-merror:0.03

[1512]	test-merror:0.034939
[1513]	test-merror:0.034905
[1514]	test-merror:0.034884
[1515]	test-merror:0.034918
[1516]	test-merror:0.034898
[1517]	test-merror:0.034891
[1518]	test-merror:0.034925
[1519]	test-merror:0.03487
[1520]	test-merror:0.03487
[1521]	test-merror:0.034905
[1522]	test-merror:0.034918
[1523]	test-merror:0.034932
[1524]	test-merror:0.034946
[1525]	test-merror:0.034946
[1526]	test-merror:0.034946
[1527]	test-merror:0.034967
[1528]	test-merror:0.03498
[1529]	test-merror:0.034946
[1530]	test-merror:0.035015
[1531]	test-merror:0.034946
[1532]	test-merror:0.035001
[1533]	test-merror:0.034987
[1534]	test-merror:0.03496
[1535]	test-merror:0.034939
[1536]	test-merror:0.034911
[1537]	test-merror:0.034884
[1538]	test-merror:0.034891
[1539]	test-merror:0.034863
[1540]	test-merror:0.03487
[1541]	test-merror:0.034884
[1542]	test-merror:0.034884
[1543]	test-merror:0.034877
[1544]	test-merror:0.034898
[1545]	test-merror:0.034905
[1546]	test-merror:0.03487
[1547]	test-merror:0.03494

[1806]	test-merror:0.034078
[1807]	test-merror:0.034017
[1808]	test-merror:0.034023
[1809]	test-merror:0.033982
[1810]	test-merror:0.033955
[1811]	test-merror:0.033982
[1812]	test-merror:0.033961
[1813]	test-merror:0.033968
[1814]	test-merror:0.033982
[1815]	test-merror:0.033961
[1816]	test-merror:0.033961
[1817]	test-merror:0.033824
[1818]	test-merror:0.033851
[1819]	test-merror:0.033893
[1820]	test-merror:0.033872
[1821]	test-merror:0.033817
[1822]	test-merror:0.033831
[1823]	test-merror:0.033906
[1824]	test-merror:0.03392
[1825]	test-merror:0.033886
[1826]	test-merror:0.033899
[1827]	test-merror:0.033893
[1828]	test-merror:0.033838
[1829]	test-merror:0.033879
[1830]	test-merror:0.033879
[1831]	test-merror:0.033886
[1832]	test-merror:0.033927
[1833]	test-merror:0.033879
[1834]	test-merror:0.033886
[1835]	test-merror:0.033858
[1836]	test-merror:0.033865
[1837]	test-merror:0.033879
[1838]	test-merror:0.033899
[1839]	test-merror:0.033879
[1840]	test-merror:0.033872
[1841]	test-merror:0.

In [ ]:
%%time
# Repeat for CPU algorithm
tmp = time.time()
param['tree_method'] = 'hist'
cpu_res = {}
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=cpu_res)
print("CPU Training Time: %s seconds" % (str(time.time() - tmp)))